Multiple Linear Regression

In [51]:
from warnings import filterwarnings
filterwarnings('ignore')

In [52]:
path = r"C:\Users\admin\Downloads\Dataset for project.csv"
import pandas as pd
df = pd.read_csv(path,na_values=['','NA'],keep_default_na=False)
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


Basic data quality checks

In [53]:
df.shape

(197428, 16)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [55]:
## Missing values
df.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

In [56]:
df.duplicated().sum()

np.int64(0)

In [57]:
df.drop_duplicates()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,00:19:41,01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0
197424,1.0,00:01:59,00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0
197425,1.0,04:46:08,05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0
197426,1.0,18:18:15,19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0


Convert the time columns to datetime

In [58]:
df ['created_at']=pd.to_datetime(df ['created_at'])
df ['actual_delivery_time']=pd.to_datetime(df ['actual_delivery_time'])

Extract time based features


Create a new column for actual time taken for delivery taken in seconds 

In [59]:
df['delivery_duration_seconds']=(df['actual_delivery_time']-df['created_at']).dt.total_seconds()

extract date and time components

In [60]:
df ['created_date']= df ['created_at'].dt.date
df ['created_time']= df ['created_at'].dt.time
df ['delivery_date']=df ['actual_delivery_time'].dt.date
df ['delivery_time']=df ['actual_delivery_time'].dt.time

There are  missing values but no  an duplicated rows in the data
Separate X and Y features
 
X : REMAINING df FEATURES except delivery_duration_seconds
Y : delivery_duration_seconds

In [61]:
X = df.drop(columns=['delivery_duration_seconds','created_at','actual_delivery_time'])
Y = df[['delivery_duration_seconds']]

In [62]:
X.dtypes

market_id                                       float64
store_id                                          int64
store_primary_category                           object
order_protocol                                  float64
total_items                                       int64
subtotal                                          int64
num_distinct_items                                int64
min_item_price                                    int64
max_item_price                                    int64
total_onshift_dashers                           float64
total_busy_dashers                              float64
total_outstanding_orders                        float64
estimated_order_place_duration                    int64
estimated_store_to_consumer_driving_duration    float64
created_date                                     object
created_time                                     object
delivery_date                                    object
delivery_time                                   

In [63]:

Y

,delivery_duration_seconds
0,3779.0
1,4024.0
2,1781.0
3,3075.0
4,2390.0
...,...
197423,3907.0
197424,3383.0
197425,3008.0
197426,3907.0


Separate categorical and continuous columns

In [64]:
cat = list(X.columns[X.dtypes=='object'])
con = list(X.columns[X.dtypes!='object'])
print(cat,con,sep='\n')

['store_primary_category', 'created_date', 'created_time', 'delivery_date', 'delivery_time']
['market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration']


Data preprocessing and Data Cleaning: 1.One hot encoding=> categorical data 2.Standard scale=> continuous data

In [78]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [79]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
OrdinalEncoder() # feature scaling
)


In [80]:
con_pipe = make_pipeline(
    SimpleImputer(strategy="mean"), # data cleaning
    StandardScaler()
)

In [81]:
pre= ColumnTransformer([
    ("cat",cat_pipe,cat),
    ("con",con_pipe,con)
]).set_output(transform='pandas')

In [82]:
pre

,transformers,"[('cat', ...), ('con', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None


In [83]:
X_pre= pre.fit_transform (X)

In [76]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit (X_pre,Y)

ValueError: Input y contains NaN.

In [ ]:
model.intercept_

NameError: name 'model' is not defined

In [ ]:
ypreds = model.predict(X_pre)
ypreds

NameError: name 'X_pre' is not defined

Evaluate the model
Mean Sqaured Error
Mean Absolute Error
RMSE
R2 score- Coeff of determination

In [73]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [74]:
mse = mean_squared_error(Y,ypreds)
mae = mean_absolute_error(Y,ypreds)
rmse = mse**(1/2)
r2 = r2_score(Y,ypreds)

print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 score: {r2*100:.2f}%")

NameError: name 'ypreds' is not defined